In [1]:
from CPPN import CPPN, geno_to_pheno, geno_to_pheno_structual
import random
import numpy as np
import open3d as o3d

# # for test stability, we need to make sure the network is deterministic
# # when realistcally used, we should eliminate the deterministic part
random.seed(4122)
np.random.seed(4122)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def calc_node_state(network, node_name, orig_size_xyz, density=5.):
    """Propagate input values through the network"""
    if network.graph.nodes[node_name]["evaluated"]:
        return network.graph.nodes[node_name]["state"]

    network.graph.nodes[node_name]["evaluated"] = True
    input_edges = network.graph.in_edges(nbunch=[node_name])
    
    flattened_size = network.num_points
    new_state = np.zeros(flattened_size)

    for edge in input_edges:
        node1, node2 = edge
        ### recursively evaluate the input node, if it hasn't been evaluated already
        ### at first, all the nodes are un-evaluated except the input nodes, i.e., 'x', 'y', 'z', 'd', 'bias'
        new_state += calc_node_state(network, node1, orig_size_xyz, density) * network.graph.edges[node1, node2]["weight"]

    network.graph.nodes[node_name]["state"] = new_state

    return network.graph.nodes[node_name]["function"](new_state)

In [3]:
cppn = CPPN(output_node_names=["out"])
shape = (1, 1, 1)
density = 10
cppn.set_input_node_states(shape, density, mode="random")
output_state = calc_node_state(cppn, "out", shape, density)
pointcloud = cppn.pointcloud

In [4]:
output_state[:5]

array([-0.28198502, -0.38022814, -0.07981278,  0.24852019,  0.26003625])

In [5]:
split_parts = geno_to_pheno(pointcloud, output_state, [0], border_width=0.05, inside_sparsity=1)

In [6]:
len(split_parts)

2

In [7]:
# # o3d visualization
color_parts = [[1, 0, 0], [0, 1, 0], [30/255.,144/255.,1], [1, 1, 1]]
# material_color_parts = [0.5, 0.5, 0.5]
material_color_parts = [1, 1, 1]
clouds = []
for i, split_part in enumerate(split_parts):
    print("material ", str(i+1), ": ", len(split_part))
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(split_part[:, :3])
    colors = np.zeros([split_part.shape[0], 3])
    colors[:, :] = color_parts[i]
    colors[split_part[:, -1] == 1, :] = material_color_parts # material points
    cloud.colors = o3d.utility.Vector3dVector(colors)
    clouds.append(cloud)
# border_points = split_parts["border"][0][0]
cloud = o3d.geometry.PointCloud()
# colors = np.zeros([border_points.shape[0], 3])
colors[:, :] = [0, 0, 1]
# cloud.points = o3d.utility.Vector3dVector(border_points)
cloud.colors = o3d.utility.Vector3dVector(colors)
clouds.append(cloud)
o3d.visualization.draw_geometries(clouds)

material  1 :  646
material  2 :  354
